# Portfolio Analysis

In this project, we seek to run through some common portfolio construction and asset allocation strategies to gain some insights into the usefulness of standard measures like sharpe ratio. The end goal is to test such strategies to a sufficient level of rigour that we can convincingly employ them into our portfolio rebalancing pipeline. I will start off by testing my own portfolio and the procedures should be generalizable. Let's get started!

## 0. Project Setup

##### Assets

1. Sea Limited (NYSE:'**SE**')
2. Bilibili Inc (NASDAQ:'**BILI**')
3. Pinduoduo Inc (NASDAQ:'**PDD**')
4. NVIDIA Corporation (NASDAQ:'**NVDA**') 
5. Unity Software Inc (NYSE:'**U**')
6. Peloton Inc (NASDAQ:'**PTON**')
7. Alphabet Inc (NASDAQ:'**GOOG**)
8. Amazon.com Inc (NASDAQ:'**AMZN**)
9. Microsoft Corporation (NASDAQ:'**MSFT**')
10. Facebook Inc (NASDAQ:'**FB**')
11. Intel Corporation (NASDAQ: '**INTC**')
12. Roku Inc (NASDAQ:'**ROKU**')
13. Roblox Corp (NYSE:'**RBLX**')
14. Blackrock Inc (NYSE:'**BLK**')
15. Berkshire Hathaway Inc (NYSE:'**BRK-B**')
16. Intellia Therapeutics Inc (NASDAQ:'**NTLA**')
17. Autodesk, Inc. (NASDAQ:'**ADSK**')
18. Draftkings Inc (NASDAQ:'**DKNG**')
19. Clearpoint Neuro Inc (NASDAQ:'**CLPT**')

##### Data

Daily adjusted closing prices from yahoo finance API 

##### Time period

1 year, 3 year, 5 year & 10 year.

In [1]:
# Load the required packages 
# Computation
import numpy as np 
from scipy import fftpack
# Plotting
import matplotlib.pyplot as plt
import mplcursors
import matplotlib.ticker as mtick
import seaborn as sns
# Data analysis
import pandas as pd
from sklearn import preprocessing
# Data source
import yfinance as yf

## 1. Data Analysis

In [3]:
# shortlisted stocks for portfolio analysis
s_list = 'SE BILI PDD NVDA PTON U INTC GOOG AMZN ROKU MSFT FB BLK BRK-B RBLX ADSK DKNG CLPT'
df_1y = yf.download(tickers = s_list, period = '1y', interval = '1d', group_by = 'ticker')
df_3y = yf.download(tickers = s_list, period = '3y', interval = '1d', group_by = 'ticker')
df_5y = yf.download(tickers = s_list, period = '5y', interval = '1d', group_by = 'ticker')
df_10y = yf.download(tickers = s_list, period = '10y', interval = '1d', group_by = 'ticker')

# Benchmark - S&P500
bench = '^GSPC'
bench_1y = yf.download(tickers = bench, period = '1y', interval = '1d')
bench_3y = yf.download(tickers = bench, period = '3y', interval = '1d')
bench_5y = yf.download(tickers = bench, period = '5y', interval = '1d')
bench_10y = yf.download(tickers = bench, period = '10y', interval = '1d')

[*********************100%***********************]  18 of 18 completed
[*********************100%***********************]  18 of 18 completed
[*********************100%***********************]  18 of 18 completed
[*********************100%***********************]  18 of 18 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [11]:
# select column for adjusted close prices
df_close_1y = df_1y.xs('Adj Close', level = 1, axis = 1)
df_close_3y = df_3y.xs('Adj Close', level = 1, axis = 1)
df_close_5y = df_5y.xs('Adj Close', level = 1, axis = 1)
df_close_10y = df_10y.xs('Adj Close', level = 1, axis = 1)
bench_close_1y = pd.DataFrame(data = bench_1y['Adj Close'], columns = ['Adj Close'])
bench_close_3y = pd.DataFrame(data = bench_3y['Adj Close'], columns = ['Adj Close'])
bench_close_5y = pd.DataFrame(data = bench_5y['Adj Close'], columns = ['Adj Close'])
bench_close_10y = pd.DataFrame(data = bench_10y['Adj Close'], columns = ['Adj Close'])

# adding column name to benchmark table
bench_close_1y = bench_close_1y.rename(columns = {'Adj Close':'S&P500'})
bench_close_3y = bench_close_3y.rename(columns = {'Adj Close':'S&P500'})
bench_close_5y = bench_close_5y.rename(columns = {'Adj Close':'S&P500'})
bench_close_10y = bench_close_10y.rename(columns = {'Adj Close':'S&P500'})
 
# reordering column names in portfolio 
s_order = ['SE', 'BILI', 'PDD', 'NVDA', 'U', 'PTON', 'GOOG', 'AMZN', 'MSFT', 'FB', 'INTC', 'ROKU', 'RBLX',  'BLK', 'BRK-B', 'ADSK', 'DKNG', 'CLPT']
df_close_1y = df_close_1y[s_order]
df_close_3y = df_close_3y[s_order]
df_close_5y = df_close_5y[s_order]
df_close_10y = df_close_10y[s_order]

# appending benchmark prices to portfolio dataframe 
df_close_1y = pd.concat([df_close_1y,bench_close_1y], axis = 1)
df_close_3y = pd.concat([df_close_3y,bench_close_3y], axis = 1)
df_close_5y = pd.concat([df_close_5y,bench_close_5y], axis = 1)
df_close_10y = pd.concat([df_close_10y,bench_close_10y], axis = 1)

# drop row if all values are NaN
df_close_1y.dropna(axis = 0, how = 'all', inplace = True)
df_close_3y.dropna(axis = 0, how = 'all', inplace = True)
df_close_5y.dropna(axis = 0, how = 'all', inplace = True)
df_close_10y.dropna(axis = 0, how = 'all', inplace = True)

# check the last 5 trading days
df_close_1y.tail(5)

,SE,BILI,PDD,NVDA,U,PTON,GOOG,AMZN,MSFT,FB,INTC,ROKU,RBLX,BLK,BRK-B,ADSK,DKNG,CLPT,S&P500
Date,,,,,,,,,,,,,,,,,,,
2021-08-16,290.489990,69.459999,82.220001,199.500000,122.190002,110.000000,2778.320068,3298.989990,294.037109,366.559998,53.470001,356.579987,79.570000,924.890015,289.450012,330.179993,52.459999,17.889999,4472.939941
2021-08-17,308.269989,67.779999,79.449997,194.580002,120.779999,113.250000,2746.010010,3241.959961,292.519989,358.450012,52.689999,357.040009,78.680000,909.619995,289.100006,324.700012,51.110001,17.129999,4448.080078
2021-08-18,318.609985,68.459999,80.959999,190.399994,117.919998,110.650002,2731.399902,3201.219971,290.730011,355.450012,52.189999,344.720001,83.459999,901.909973,286.359985,323.519989,52.099998,17.600000,4400.270020
2021-08-19,321.119995,64.330002,74.690002,197.979996,116.410004,107.800003,2738.270020,3187.750000,296.769989,355.119995,52.439999,343.350006,81.349998,903.780029,285.130005,330.350006,51.540001,17.020000,4405.799805
2021-08-20,316.309692,68.894997,79.629997,204.220001,121.419998,109.570000,2761.810059,3202.901855,305.515015,358.149994,52.195000,348.929993,83.959999,914.760010,285.375000,334.867493,51.889999,16.799999,4435.319824


In [141]:
# normalize the prices 
df_close_1y_start = df_close_1y.iloc[0]
df_close_1y_norm = (df_close_1y - df_close_1y_start)/df_close_1y_start

df_close_3y_start = df_close_3y.iloc[0]
df_close_3y_norm = (df_close_3y - df_close_3y_start)/df_close_3y_start

df_close_5y_start = df_close_5y.iloc[0]
df_close_5y_norm = (df_close_5y - df_close_5y_start)/df_close_5y_start

df_close_10y_start = df_close_10y.iloc[0]
df_close_10y_norm = (df_close_10y - df_close_10y_start)/df_close_10y_start

# plotting out the prices
%matplotlib widget

# plot configurations
sns.set(style="darkgrid", font_scale=0.8)
palette = sns.color_palette("hls", 19)
fig, ax = plt.subplots(figsize=(8, 4))

# plotting out the figure
plot1 = sns.lineplot(ax=ax, data = df_close_1y_norm, dashes = False, palette=palette)
plt.legend(bbox_to_anchor=(1, 1), loc=2, borderaxespad=0., fontsize = 8)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.title('1 Year Returns')

# make it interactive
cursor = mplcursors.cursor(plot1, hover=True)
@cursor.connect("add")
def on_add(sel):
    sel.annotation.set(text=tt[sel.target.index])
    
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
# check the dimensions 
df_close_1y.shape

(253, 19)

In [142]:
# convert the price into rate of return
df_return_1y = df_close_1y.pct_change()
df_return_3y = df_close_3y.pct_change()
df_return_5y = df_close_5y.pct_change()
df_return_10y = df_close_10y.pct_change()

# check the descriptive statistics of 1 year rate of return
df_return_1y.describe()

,SE,BILI,PDD,NVDA,U,PTON,GOOG,AMZN,MSFT,FB,INTC,ROKU,RBLX,BLK,BRK-B,ADSK,DKNG,CLPT,S&P500
count,252.000000,252.000000,252.000000,252.000000,232.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,114.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000
mean,0.003526,0.002806,0.000234,0.002428,0.003370,0.002797,0.002351,0.000050,0.001560,0.001355,0.000556,0.004153,0.002679,0.001988,0.001353,0.001368,0.002171,0.006885,0.001117
std,0.035514,0.047680,0.045761,0.026768,0.042341,0.043267,0.016622,0.018197,0.015516,0.020999,0.020720,0.039628,0.045862,0.015176,0.011082,0.019666,0.039216,0.051330,0.009446
min,-0.103195,-0.170397,-0.135180,-0.092775,-0.141312,-0.202853,-0.054630,-0.075649,-0.061947,-0.063099,-0.105751,-0.124132,-0.123584,-0.046489,-0.029872,-0.073694,-0.084906,-0.165829,-0.035288
25%,-0.017305,-0.027241,-0.026236,-0.011579,-0.024125,-0.020480,-0.005608,-0.009493,-0.005926,-0.011873,-0.010037,-0.019454,-0.027131,-0.007026,-0.005363,-0.009308,-0.023648,-0.023929,-0.003660
50%,0.005112,0.001163,-0.002933,0.003151,0.000207,0.003608,0.002408,0.000772,0.001299,-0.000310,0.000514,0.000666,0.000093,0.002630,0.001405,0.002723,-0.000422,0.001658,0.001266
75%,0.026251,0.031729,0.021232,0.018631,0.029478,0.028578,0.010171,0.010112,0.010940,0.014377,0.011889,0.023310,0.029491,0.011605,0.007806,0.014797,0.023732,0.034429,0.007155
max,0.108809,0.221735,0.204109,0.080333,0.162555,0.144746,0.073961,0.063230,0.048249,0.083227,0.069684,0.176669,0.213281,0.040412,0.060570,0.047380,0.172697,0.164811,0.023791


### Rate of Return

In [147]:
# compute the annualized average rate of return across a 1 year, 3 year, 5 year & 10 year horizon.
nday = 252 # number of trading days
df_return_1y_mean = pd.DataFrame(data = df_return_1y.mean(axis = 0), columns = ['1Y Annual Return']) * nday
df_return_3y_mean = pd.DataFrame(data = df_return_3y.mean(axis = 0), columns = ['3Y Annual Return']) * nday
df_return_5y_mean = pd.DataFrame(data = df_return_5y.mean(axis = 0), columns = ['5Y Annual Return']) * nday
df_return_10y_mean = pd.DataFrame(data = df_return_10y.mean(axis = 0), columns = ['10Y Annual Return']) * nday
data_frames = [df_return_1y_mean, df_return_3y_mean, df_return_5y_mean, df_return_10y_mean]
df_return_merge = pd.concat(data_frames, axis = 1)

#df_return_10y_mean.plot.barh()
df_return_merge

,1Y Annual Return,3Y Annual Return,5Y Annual Return,10Y Annual Return
SE,0.888446,1.218564,0.949258,0.949258
BILI,0.707109,0.804302,0.744997,0.744997
PDD,0.059061,0.728437,0.607848,0.607848
NVDA,0.611935,0.529104,0.633075,0.515353
U,0.849225,0.849225,0.849225,0.849225
PTON,0.704743,1.013514,1.013514,1.013514
GOOG,0.592515,0.322416,0.291020,0.273480
AMZN,0.012588,0.232101,0.331536,0.338233
MSFT,0.393040,0.412450,0.386965,0.309525
FB,0.341356,0.308982,0.266131,0.309584


We see that the annualized returns are skewed to the right due to the broad spike in equity valuations in 2020. Also, some of the companies in the list only went public in the last 5 years, which explains the potentially transitory outperformance against the benchmark. 

### Variance

In [144]:
# compute the daily standard deviation across a 1 year, 3 year, 5 year & 10 year horizon.
df_std_1y = pd.DataFrame(data = df_close_1y_norm.std(axis = 0), columns = ['1Y Daily Standard Deviation'])  
df_std_3y = pd.DataFrame(data = df_close_3y_norm.std(axis = 0), columns = ['3Y Daily Standard Deviation']) 
df_std_5y = pd.DataFrame(data = df_close_5y_norm.std(axis = 0), columns = ['5Y Daily Standard Deviation']) 
df_std_10y = pd.DataFrame(data = df_close_10y_norm.std(axis = 0), columns = ['10Y Daily Standard Deviation']) 
data_frames = [df_std_1y, df_std_3y, df_std_5y, df_std_10y]
df_std_merge = pd.concat(data_frames, axis = 1)

# compute the daily variance across a 1 year, 3 year, 5 year & 10 year horizon.
df_var_1y = pd.DataFrame(data = df_close_1y_norm.var(axis = 0), columns = ['1Y Daily Variance'])  
df_var_3y = pd.DataFrame(data = df_close_3y_norm.var(axis = 0), columns = ['3Y Daily Variance']) 
df_var_5y = pd.DataFrame(data = df_close_5y_norm.var(axis = 0), columns = ['5Y Daily Variance']) 
df_var_10y = pd.DataFrame(data = df_close_10y_norm.var(axis = 0), columns = ['10Y Daily Variance']) 
data_frames = [df_var_1y, df_var_3y, df_var_5y, df_var_10y]
df_var_merge = pd.concat(data_frames, axis = 1)

#df_return_10y_mean.plot.barh()
df_var_merge

,1Y Daily Variance,3Y Daily Variance,5Y Daily Variance,10Y Daily Variance
SE,0.098202,46.083547,NaN,NaN
BILI,0.492311,11.337485,NaN,NaN
PDD,0.124702,6.888899,NaN,NaN
NVDA,0.043753,0.629571,8.688940,273.949829
U,NaN,NaN,NaN,NaN
PTON,0.090376,NaN,NaN,NaN
GOOG,0.064429,0.149371,0.356167,4.453960
AMZN,0.002415,0.138055,1.262220,30.501824
MSFT,0.015214,0.286299,1.496730,12.404332
FB,0.016163,0.123361,0.222230,NaN


In [126]:
# compute the pearson pairwise correlation matrix
df_return_1y_corr = df_return_1y.corr(method='pearson')
df_return_3y_corr = df_return_3y.corr(method='pearson')
df_return_5y_corr = df_return_5y.corr(method='pearson')
df_return_10y_corr = df_return_10y.corr(method='pearson')

In [127]:
# plotting the 1 year correlation matrix 

sns.set_theme(style="white")
# generate a mask for the upper triangle
mask = np.triu(np.ones_like(df_return_1y_corr, dtype=bool))

# set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# generate a custom diverging colormap
cmap = sns.diverging_palette(240, 5, as_cmap=True)

# draw the heatmap with the mask and correct aspect ratio
sns.heatmap(df_return_1y_corr, mask=mask, cmap=cmap, vmax=1, center=0, vmin = -1,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

This result is sort of expected. In general, we know that the big tech should be highly correlated with the S&P500, and the so-called "high-beta/ARK" stocks should correlate with each other. BRK-B has a "defensive" reputation so it makes sense that it would have zero to negative correlation with the mid-cap tech stocks. Lastly, we note that the textbook diversification of idiosyncratic risks seems elusive within an all equity portfolio.

In [151]:
# compute the pairwise variance covariance matrix 
df_return_1y_cov = df_return_1y.cov()
df_return_3y_cov = df_return_3y.cov()
df_return_5y_cov = df_return_5y.cov()
df_return_10y_cov = df_return_10y.cov()

df_return_1y_cov

,SE,BILI,PDD,NVDA,U,PTON,GOOG,AMZN,MSFT,FB,INTC,ROKU,RBLX,BLK,BRK-B,ADSK,DKNG,CLPT,S&P500
SE,0.001261,0.000940,0.000949,0.000511,0.000688,0.000582,0.000210,0.000282,0.000249,0.000243,0.000224,0.000726,0.000625,0.000141,-0.000016,0.000367,0.000456,0.000482,0.000133
BILI,0.000940,0.002273,0.001288,0.000436,0.000711,0.000492,0.000189,0.000218,0.000163,0.000222,0.000149,0.000714,0.000616,0.000101,-0.000011,0.000317,0.000413,0.000437,0.000114
PDD,0.000949,0.001288,0.002094,0.000397,0.000655,0.000402,0.000238,0.000262,0.000224,0.000284,0.000190,0.000517,0.000731,0.000136,-0.000004,0.000285,0.000397,0.000426,0.000120
NVDA,0.000511,0.000436,0.000397,0.000717,0.000401,0.000448,0.000222,0.000292,0.000278,0.000302,0.000221,0.000476,0.000441,0.000164,0.000020,0.000325,0.000281,0.000230,0.000150
U,0.000688,0.000711,0.000655,0.000401,0.001793,0.000663,0.000112,0.000308,0.000225,0.000234,0.000122,0.000803,0.000639,0.000098,-0.000053,0.000279,0.000509,0.000585,0.000098
PTON,0.000582,0.000492,0.000402,0.000448,0.000663,0.001872,0.000076,0.000255,0.000196,0.000177,0.000169,0.000867,0.000486,0.000101,-0.000084,0.000252,0.000347,0.000352,0.000068
GOOG,0.000210,0.000189,0.000238,0.000222,0.000112,0.000076,0.000276,0.000185,0.000184,0.000235,0.000116,0.000180,0.000116,0.000126,0.000058,0.000176,0.000140,0.000109,0.000115
AMZN,0.000282,0.000218,0.000262,0.000292,0.000308,0.000255,0.000185,0.000331,0.000206,0.000254,0.000125,0.000287,0.000186,0.000098,0.000013,0.000182,0.000206,0.000113,0.000107
MSFT,0.000249,0.000163,0.000224,0.000278,0.000225,0.000196,0.000184,0.000206,0.000241,0.000201,0.000121,0.000237,0.000114,0.000120,0.000036,0.000190,0.000158,0.000156,0.000111
FB,0.000243,0.000222,0.000284,0.000302,0.000234,0.000177,0.000235,0.000254,0.000201,0.000441,0.000110,0.000302,0.000256,0.000128,0.000035,0.000175,0.000158,0.000158,0.000121


### Risk-Return Measures

In [129]:
# Sharpe ratio 
df_sharpe_merge = np.divide(df_return_merge,df_std_merge)
df_sharpe_merge

,1Y Annual Return,3Y Annual Return,5Y Annual Return,10Y Annual Return
SE,2.835117,0.179505,NaN,NaN
BILI,1.007781,0.238870,NaN,NaN
PDD,0.167249,0.277535,NaN,NaN
NVDA,2.925511,0.666836,0.214769,0.031136
U,NaN,NaN,NaN,NaN
PTON,2.344254,NaN,NaN,NaN
GOOG,2.334317,0.834224,0.487637,0.129584
AMZN,0.256171,0.624670,0.295096,0.061243
MSFT,3.186456,0.770834,0.316301,0.087884
FB,2.684978,0.879720,0.564540,NaN


## 2. Portfolio Optimization

### Current Allocation

In [130]:
# current weightage 
arr_w = np.array([[0.30,0.14,0.10,0.08,0.08,0.04,0.04,0.04,0.03,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.01,0.01]])
df_w = pd.DataFrame(data = arr_w, columns = s_order)
df_w = df_w.rename(index={0: "weight"})
df_w

,SE,BILI,PDD,NVDA,U,PTON,GOOG,AMZN,MSFT,FB,INTC,ROKU,RBLX,BLK,BRK-B,ADSK,DKNG,CLPT
weight,0.3,0.14,0.1,0.08,0.08,0.04,0.04,0.04,0.03,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.01,0.01


In [27]:
arr_w.sum()

1.0

In [131]:
df_return_1y_mean

,1Y Annual Return
SE,0.888446
BILI,0.707109
PDD,0.059061
NVDA,0.611935
U,0.849225
PTON,0.704743
GOOG,0.592515
AMZN,0.012588
MSFT,0.393040
FB,0.341356


In [138]:
df_return_1y_cov

,SE,BILI,PDD,NVDA,U,PTON,GOOG,AMZN,MSFT,FB,INTC,ROKU,RBLX,BLK,BRK-B,ADSK,DKNG,CLPT
SE,0.001261,0.000940,0.000949,0.000511,0.000688,0.000582,0.000210,0.000282,0.000249,0.000243,0.000224,0.000726,0.000625,0.000141,-0.000016,0.000367,0.000456,0.000482
BILI,0.000940,0.002273,0.001288,0.000436,0.000711,0.000492,0.000189,0.000218,0.000163,0.000222,0.000149,0.000714,0.000616,0.000101,-0.000011,0.000317,0.000413,0.000437
PDD,0.000949,0.001288,0.002094,0.000397,0.000655,0.000402,0.000238,0.000262,0.000224,0.000284,0.000190,0.000517,0.000731,0.000136,-0.000004,0.000285,0.000397,0.000426
NVDA,0.000511,0.000436,0.000397,0.000717,0.000401,0.000448,0.000222,0.000292,0.000278,0.000302,0.000221,0.000476,0.000441,0.000164,0.000020,0.000325,0.000281,0.000230
U,0.000688,0.000711,0.000655,0.000401,0.001793,0.000663,0.000112,0.000308,0.000225,0.000234,0.000122,0.000803,0.000639,0.000098,-0.000053,0.000279,0.000509,0.000585
PTON,0.000582,0.000492,0.000402,0.000448,0.000663,0.001872,0.000076,0.000255,0.000196,0.000177,0.000169,0.000867,0.000486,0.000101,-0.000084,0.000252,0.000347,0.000352
GOOG,0.000210,0.000189,0.000238,0.000222,0.000112,0.000076,0.000276,0.000185,0.000184,0.000235,0.000116,0.000180,0.000116,0.000126,0.000058,0.000176,0.000140,0.000109
AMZN,0.000282,0.000218,0.000262,0.000292,0.000308,0.000255,0.000185,0.000331,0.000206,0.000254,0.000125,0.000287,0.000186,0.000098,0.000013,0.000182,0.000206,0.000113
MSFT,0.000249,0.000163,0.000224,0.000278,0.000225,0.000196,0.000184,0.000206,0.000241,0.000201,0.000121,0.000237,0.000114,0.000120,0.000036,0.000190,0.000158,0.000156
FB,0.000243,0.000222,0.000284,0.000302,0.000234,0.000177,0.000235,0.000254,0.000201,0.000441,0.000110,0.000302,0.000256,0.000128,0.000035,0.000175,0.000158,0.000158


In [148]:
# dropping benchmark from dataframe
df_return_1y_mean = df_return_1y_mean.drop(index = ['S&P500'])

df_return_1y_cov = df_return_1y_cov.drop(columns= ['S&P500'])
df_return_1y_cov = df_return_1y_cov.drop(index= ['S&P500'])

In [149]:
# convert dataframe back to numpy array for ease of manipulation
np_return_1y_mean = df_return_1y_mean.T.to_numpy()

np_return_1y_cov = df_return_1y_cov.to_numpy()

In [136]:
df_w

,SE,BILI,PDD,NVDA,U,PTON,GOOG,AMZN,MSFT,FB,INTC,ROKU,RBLX,BLK,BRK-B,ADSK,DKNG,CLPT
weight,0.3,0.14,0.1,0.08,0.08,0.04,0.04,0.04,0.03,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.01,0.01


In [150]:
# portfolio return 
portfolio_return_1y = np.inner(np_return_1y_mean,arr_w)
portfolio_var_1y = np.matmul(np.matmul(arr_w,np_return_1y_cov),arr_w.T)
print(f'Portfolio 1y return:{portfolio_return_1y}; Portfolio 1y variance:{portfolio_var_1y}')

Portfolio 1y return:[[0.63968263]]; Portfolio 1y variance:[[0.00062805]]


### Minimum Variance Portfolio 

Given a portfolio of $n$ assets and a required portfolio rate of return $\mu_{p}$, find $w_{i}$ for $i \in [1:n]$ such that the portfolio variance $\sigma_{p}$ is minimized. 

### Maximum Return Portfolio

Given a portfolio of $n$ assets and a maximum portfolio variance  $\sigma_{p}$, find $w_{i}$ for $i \in [1:n]$ such that the portfolio rate of return $\mu_{p}$ is minimized. 